# WELL NOTEBOOK
## Well logs visualization &amp; petrophysics

Install the the repository reservoirpy from github and import the required packages

In [1]:
import os
path = os.path.abspath(os.path.join('..','..','..'))
import sys
sys.path.append(path)
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium
from pyproj import Proj, transform, CRS, Transformer

from reservoirpy.wellpy import path as ph


### Well atributes

Well atributes, name, rte, coordinates, survey

In [2]:
#Create the well object
name1 = 'well-1'
rte1 = 515.78 # Rotary table Elevation
surf_coord1 = Point(1000100,1000000,520)
crs1 = 'EPSG:3117'

w1 = ph.well(name=name1, 
             rte=rte1, 
             surf_coord=surf_coord1, 
             crs=crs1)


#Create the well object
name2 = 'well-2'
rte2 = 845.2 # Rotary table Elevation
surf_coord2 = Point(1000510,1000025,500)
crs2 = 'EPSG:3117'

w2 = ph.well(name=name2, 
             rte=rte2, 
             surf_coord=surf_coord2, 
             crs=crs2)


#Create the well object
name3 = 'well-3'
rte3 = 245.2 # Rotary table Elevation
surf_coord3 = Point(1000054,1000028,500)
crs3 = 'EPSG:3117'

w3 = ph.well(name=name3, 
             rte=rte3, 
             surf_coord=surf_coord3, 
             crs=crs3)

## Create an empty wells group

In [3]:
g1 = ph.wells_group()

In [4]:
g1.add_well(w1,w2,w3)
g1.wells

{'well-1': <reservoirpy.wellpy.path.wells.well at 0x7f5a64b312d0>,
 'well-2': <reservoirpy.wellpy.path.wells.well at 0x7f5a64b313d0>,
 'well-3': <reservoirpy.wellpy.path.wells.well at 0x7f5a64b31510>}

In [5]:
dist = g1.wells_distance(wells=None,z=True,z_unit='ft')
dist

,well-1,well-2,well-3
well-1,0.000000,410.806720,54.195583
well-2,410.806720,0.000000,456.009868
well-3,54.195583,456.009868,0.000000


In [7]:
m = g1.wells_map(zoom=20)
m